# Momentumwindow excitations

In this example I will show how momentumwindows can be used to obtain more accurate excitation energies, at the cost of performance. Momentumwindows are also not well integrated into the mpskit ecosystem, so the excited states will be less "useful" in a sense.

Let's start out by finding the groundstate

In [1]:
using Revise, MPSKit, TensorKit, MPSKitExperimental, MPSKitModels, KrylovKit

th = heisenberg_XXX(SU2Irrep);
ts = InfiniteMPS(th.pspaces,[Rep[SU₂](1//2=>20,3//2=>10,5//2=>5)]);
(ts,gs_env) = find_groundstate(ts,th,VUMPS(maxiter=200,verbose=false));

And the reference energy/variance of the excitations:

In [2]:
K = 0.7
(E,A) = excitations(th,QuasiparticleAnsatz(),K,ts,sector = SU2Irrep(1));
E[1], variance(A[1],th)

(1.8896664789301174 - 7.333848733737207e-14im, 0.0027226651154292902)

You can obtain better energies by creating a momentumwindow:

In [3]:
window_length = 5
windowchi = Rep[SU₂](1//2=>20,3//2=>10,5//2=>5) # bond dimension of the window

window = LeftGaugedMW(rand,window_length,windowchi,ts,momentum =complex(K),utilspace= Rep[SU₂](1=>1));


And plopping it into dmrg!

In [4]:
(window,E) = find_groundstate(window,th,DMRG(maxiter=3))

(topit, err) = (1, 0.9997768909210545)


(topit, err) = (2, 0.13619929541167353)


(topit, err) = (3, 0.032695781805348934)


(LeftGaugedMW{TensorMap{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 2, 2, SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Matrix{ComplexF64}}, FusionTree{SU2Irrep, 2, 0, 1, Nothing}, FusionTree{SU2Irrep, 2, 0, 1, Nothing}}, FiniteMPS{TensorMap{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 2, 1, SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Matrix{ComplexF64}}, FusionTree{SU2Irrep, 2, 0, 1, Nothing}, FusionTree{SU2Irrep, 1, 0, 0, Nothing}}, TensorMap{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 1, 1, SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Matrix{ComplexF64}}, FusionTree{SU2Irrep, 1, 0, 0, Nothing}, FusionTree{SU2Irrep, 1, 0, 0, Nothing}}}, InfiniteMPS{TensorMap{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 2, 1, SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Matrix{ComplexF64}}, FusionTree{SU2Irrep, 2, 0, 1, Nothing}, FusionTree{SU2Irrep, 1, 0, 0, Nothing}}, TensorMap{GradedSpace{SU2Irrep,

Few things to note 

- this code is **experimental**
- not well integrated with the rest of mpskit (and might never be)
- call signature / return values of find_groundstate is completely different from the typical mpskit one
- a lot of basic utility functions for windows are missing completely
- it is far slower than the quasiparticle code, potentially due to some incorrect contractions
- a window length of i corresponds with a B tensor of length i+1